In [8]:
!git clone https://github.com/nsphuoc10/transformer-based.git

Cloning into 'transformer-based'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 69 (delta 20), reused 53 (delta 12), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [ ]:
%cd transformer-based/

In [10]:
!ls

Batch.py      MetricCalculationNew.py  Sublayers.py
Beam.py       Models.py		       test_result
data	      Optim.py		       Tokenize.py
Embed.py      PreProcessNew.py	       train.py
floyd.yml     Process.py	       translate.py
huongdan.txt  README.md		       translate_transformer_temp.csv
Layers.py     run_train.ipynb
LICENSE       run_translate.ipynb


In [ ]:
!python3 -m spacy download en && spacy download vi
!python3 -m pip install pyvi

In [13]:
!python3 train.py -SGDR 1 -floyd -batchsize 512 -checkpoint 20 -epochs 5 -printevery 10 -max_strlen 100 -src_data 'data/train.vi' -trg_data 'data/train.bana' -src_lang vi -trg_lang eny

torch.__version__: 1.9.0+cu102
opt: Namespace(SGDR=1, batchsize=512, checkpoint=20, create_valset=False, d_model=512, device=0, dropout=0.1, epochs=5, floyd=False, heads=8, load_weights=None, lr=0.0001, max_strlen=100, n_layers=6, printevery=10, src_data='data/train.vi', src_lang='vi', trg_data='data/train.bana', trg_lang='en')
loading spacy tokenizers...
creating dataset and iterator... 
len(SRC.vocab): 5491
len(TRG.vocab): 4296
model weights will be saved every 20 minutes and at end of epoch to directory weights/
training model...
training ep 0 - trainlen: 420
>>> time: [04:14:32] training ep: 0 - batch(i): 0 , loss: 8.514866828918457
>>> time: [04:14:32] training ep: 0 - batch(i): 10 , loss: 7.581019401550293
>>> time: [04:14:33] training ep: 0 - batch(i): 20 , loss: 7.218713283538818
>>> time: [04:14:34] training ep: 0 - batch(i): 30 , loss: 6.8856706619262695
>>> time: [04:14:34] training ep: 0 - batch(i): 40 , loss: 6.757466793060303
>>> time: [04:14:35] training ep: 0 - batch(i)

In [ ]:
!cat translate.py

In [25]:
!python3 translate.py -load_weights weights -src_lang vi -trg_lang en

loading spacy tokenizers...
loading presaved fields...
opt.load_weights weights
loading pretrained weights...
[VI] So với kết quả tuyển dụng của người trúng tuyển đã bị hủy kết quả trúng tuyển

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
postProcessing So với kết quả tuyển dụng của người trúng tuyển đã bị hủy kết quả trúng tuyển Pơting adring thu 'yŏk tueân juêng ñôêng troê tueân troê troê tueân troê tueân thoi qua troê tueân troê tueân
[BANA] Pơting adring thu 'yŏk tueân juêng ñôêng troê tueân

In [24]:
!touch new_test_result/test-sentences-vi.txt
!touch new_test_result/test-sentences-bana.txt
!touch new_test_result/truth-sentences-bana.txt

In [27]:
import warnings
from collections import Counter

from nltk import ngrams
warnings.filterwarnings('ignore')
import nltk.translate.bleu_score as bleu


def calculateBleuScore(input_dir="data", output_dir="bleu_calculation", ref_corpus="", candidate_corpus=""):
    txtRefFile = open(input_dir + "/" + ref_corpus, encoding='utf-8', errors='ignore').read()
    txtCandidateFile = open(input_dir + '/' + candidate_corpus, encoding='utf-8', errors='ignore').read()
    txtRefFile = txtRefFile.replace("!\n", "\n").replace("?\n", "\n").replace(".\n", "\n").replace(":\n", "\n")
    txtRefFile = txtRefFile.replace("(", "").replace(")", "")

    txtCandidateFile = txtCandidateFile.replace("!\n", "\n").replace("?\n", "\n").replace(".\n", "\n").replace(":\n",                                                                                                           "\n")
    txtCandidateFile = txtCandidateFile.replace(" đ c ", "đ/c")
    splitsRef = txtRefFile.split('\n')
    splitsCandidate = txtCandidateFile.split('\n')
    print(splitsRef[1].split())
    print(splitsCandidate[1].split())
    # score = bleu_score(splitsCandidate[0].split(), splitsRef[0].split())
    # score = bleu.sentence_bleu([splitsRef[2].split()], splitsCandidate[2].split())
    # hypothesis1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which']
    # reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that']
    # reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which']
    # score = bleu.sentence_bleu([reference1, reference2], hypothesis1)
    # print('calculateBleuScore: ', score)
    # temp = list(map(lambda ref: [ref.split()], splitsRef))
    # print(temp)
    splC = list(map(lambda candidate: candidate.split(), splitsCandidate))
    score = bleu.corpus_bleu(list(map(lambda ref: [ref.split()], splitsRef)), splC)
    print('Bleu score with 200 samples is ', score)

    with open(input_dir + '/' + ref_corpus + '.t', 'w', encoding='utf-8') as f:
        f.write(txtRefFile)


# ttt = postProcessing(src_sentence="Bố đang đan bảy cái nữa.", translate_sentence="bă atung wă tanh tơpơh tŏ piêu. ....")
# print('post processing: ', ttt)
calculateBleuScore(input_dir='new_test_result', ref_corpus="truth-sentences-bana.txt",
                   candidate_corpus="test-sentences-bana.txt")

['Jônang', 'oeêng', 'dônoê', 'pôm', 'chô', 'lômo:', 'Sônaêm', "'baêl", 'ôbôêu', 'khoâng', 'hrieâng', 'minh', 'jiêt', 'tôpôêh', 'pôm', 'minh', 'chô', 'lômo', 'dônoê', 'xa', 'Nhôn', 'Loâk,', 'thi', 'xa', 'An', 'Nhôn,', 'ngoê', 'hnheêi', 'dônoê', 'oeêng', 'pôm', 'chô', 'lômo', 'maê', "'baêl", 'tômöt', 'sônaêm', "'baêl", 'ôbôêu', 'khoâng', 'hrieâng', 'minh', 'jiêt', 'tôhngam']
['Jônang', 'oeêng', 'dônoê', 'tôjung', 'pôm', 'chô', 'lômo', 'sônaêm', "'baêl", 'ôbôêu', 'khoâng', 'hrieâng', 'minh', 'jiêt', 'pôñaêm', 'tôjung', 'pôm', 'maêng', 'maê', "'baêl", 'ôbôêu', 'khoâng', 'hrieâng', 'minh', 'jiêt', 'pôñaêm', 'lôêm', 'xa', 'thi', 'xa', 'an', 'nhôn', 'khao', 'sat', 'oeêng', 'dônoê', 'tôjung', 'pôm', 'chô', 'lômo', "'baêl", 'maêng', 'maê', 'minh', 'ñôêng', "'baêl", 'sônaêm', "'baêl", 'ôbôêu', 'khoâng', 'hrieâng', 'minh', 'jiêt', 'tôhngam', 'Ñieâu.', 'tôdrong', 'pôm']
Bleu score with 200 samples is  0.27622752741860107
